# BONUS: Duckdb

![DuckDB Logo](images/logos/DuckDB_Logo.png)

DuckDB is the new black in data engineering - an in-process analytical database that aims for performance. DuckDB is focused on making it easy to query data from anywhere and has bindings to most popular languages, including Python of course. It even compiles to WASM, letting us do cool stuff like [this](https://shell.duckdb.org/)

DuckDB takes advantage of Arrow as it's internal data format, making it easy to interop with popular Python libraries as DuckDB can read and write the Arrow memory directly

In [ ]:
import duckdb
import polars as pl
pl.Config().set_thousands_separator(',')

In [ ]:
sql = """SELECT * FROM 'data/10.csv' WHERE language = 'english'"""

duckdb.sql(sql).pl()

DuckDB infers that we want to read a 'csv' file and calls it's `read_csv` function implicitly. We can of course do this explicitly if we want to pass options to handle those messy CSV files.

In [ ]:
sql = """SELECT filename, * FROM read_csv('data/10.csv', filename = true) WHERE language = 'english'"""
my_polars_df = duckdb.sql(sql).pl()
my_polars_df

Because DuckDB is both in-process, as well as Arrow-backed, it's able to easily interop with other analytical tools, such as `polars` and `pandas`

In [ ]:
sql = """
SELECT CAST(received_for_free as bool) as received_for_free, 
AVG(votes_up) as num_upvotes  
FROM my_polars_df 
GROUP BY ALL
"""
duckdb.execute(sql).pl()

This works with Parquet as well, while supporting using the Parquet metadata to filter data, allowing us to process larger-than-RAM data easily

In [ ]:
sql = """
SELECT language, count() as num_languages
FROM 'data/parquet/all_reviews.parquet' 
GROUP BY ALL
ORDER BY num_languages DESC
"""
duckdb.sql(sql).pl()

## Reading remote data
A killer feature is the nativeness of reading data from object stores directly, including common data lake formats such as Parquet. It can even query MySQL and Postgres!

Duckdb comes with a built-in secrets manager to handle credentials for connecting to remote stores so lets set that up

In [ ]:
duckdb.execute("""CREATE OR REPLACE SECRET minio (
    TYPE S3,
    KEY_ID 'minio',
    SECRET 'minio1234',
    ENDPOINT 'minio:9000',
    URL_STYLE 'path',
    USE_SSL false,
    REGION 'us-east-1'
)
""");

Secrets can be stored persistently or in-memory - here we persist in-memory

In [ ]:
duckdb.execute("FROM duckdb_secrets()").pl()

With credentials in order, we can treat S3 as just another file location

In [ ]:
sql = "FROM 's3://datalake/extract/reviews/10.csv'"

duckdb.execute(sql).pl()

Since DuckDB can both read and write from remote locations in a number of file formats, it's a great swiss army knife for ETL - let's build a tiny pipeline to clean up the review data and convert to Parquet.

In [ ]:
sql = "COPY (SELECT * FROM 's3://datalake/extract/reviews/10.csv' WHERE recommendationid is not null) TO 's3://datalake/extract/duckdb/10.parquet' (FORMAT PARQUET)"
duckdb.sql(sql)

In [ ]:
sql = "SELECT language, COUNT() as num_languages FROM 's3://datalake/extract/duckdb/10.parquet' GROUP BY ALL ORDER BY num_languages DESC"
duckdb.sql(sql).pl()

DuckDB will intelligently use the S3 `Range` header to fetch only the data that is required from the Parquet file, unlike the CSV where we need to read the whole CSV file first

In [ ]:
sql = """
SELECT language, COUNT() as num_rows 
FROM 's3://datalake/extract/duckdb/all_reviews.parquet' 
GROUP BY ALL 
ORDER BY num_rows DESC
"""
duckdb.sql(sql).pl()

We can also parse multiple files using a glob - very handy for folders of data

In [ ]:
sql = """
SELECT filename.parse_filename(true) as game_id, * EXCLUDE filename
FROM read_csv('s3://datalake/extract/reviews/*.csv', filename = true)
WHERE recommendationid is not null
LIMIT 100
"""
duckdb.execute(sql).pl()

### Iceberg
Can we do this with Iceberg? Of course! Let's use the AWS data from before to show off a more common usecase. 

In [ ]:
sql = """CREATE OR REPLACE SECRET pydata (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    SCOPE 's3://pydata-copenhagen-datalake'
)
"""
duckdb.sql(sql)

DuckDB needs to know what Metadata file is the most current one, so we can use our Iceberg Catalog to get that information

In [ ]:
from pyiceberg.catalog import load_catalog

In [ ]:
catalog = load_catalog("aws_iceberg", **{"type": "glue", "glue.region": "eu-north-1"})

In [ ]:
table = catalog.load_table("steam.reviews")

In [ ]:
table.metadata_location

DuckDB is extendible via `extensions`. We have actually been using the `httpfs` extension to talk to S3, but that one is auto-loaded when used since it's a common one to use. The Iceberg extension needs to be explicitly installed and loaded

In [ ]:
duckdb.install_extension('iceberg')
duckdb.load_extension('iceberg')

In [ ]:
sql = f"SELECT COUNT() as num_reviews FROM iceberg_scan('{table.metadata_location}')"

In [ ]:
duckdb.sql(sql).pl()

### Taking it up a notch
Let's do something slightly more complicated - finding what language a given game_id has the most reviews in. We will also combine local and external data by fetching a mapping of game_id to name from the Steam API

In [ ]:
sql = "CREATE TABLE apps as (SELECT unnest(applist.apps, recursive := true) FROM read_json('https://api.steampowered.com/ISteamApps/GetAppList/v2'))"
duckdb.sql(sql)

In [ ]:
sql = f"""
with ranked_reviews as (
SELECT 
    game_id, 
    language, 
    count() as num_reviews,
    row_number() OVER (PARTITION BY game_id ORDER BY num_reviews DESC) as rank
    FROM iceberg_scan('{table.metadata_location}')
    GROUP BY game_id, language
    QUALIFY
        rank = 1
)
SELECT 
apps.name, 
language, 
num_reviews 
FROM ranked_reviews
JOIN apps on ranked_reviews.game_id = apps.appid
ORDER BY num_reviews DESC
"""
duckdb.sql(sql).pl()

This is such a common pattern, that pyiceberg by default has a `to_duckdb` method. Note, that this will download the data locally to work with in memory, so we would no longer be pushing queries to the storage backend - large datasets need to be filtered first.

In [ ]:
db = table.scan(selected_fields=["language"]).to_duckdb(table_name='languages')

In [ ]:
db.sql("""
SELECT language, count() as num_languages 
FROM languages 
GROUP BY ALL
ORDER BY num_languages DESC""").pl()